In [8]:
import finnhub
from settings import *
from datetime import datetime, timedelta, date
import pandas as pd
import requests
import time
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib
from pretty_html_table import build_table

In [9]:
def get_sp500():
    global tickers_delisted
    website = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    web_df = pd.read_html(website)[0]

    us_df = pd.DataFrame(columns=web_df.columns)
    us_list = ["RIVN", "ARM"]
    us_df["Symbol"] = us_list
    us_df["Security"] = us_list
    us_df["GICS Sector"] = "Consumer Discretionary"
    us_df["GICS Sub-Industry"] = "Automobile Manufacturers"
    web_df = pd.concat([web_df, us_df], axis=0)

    # Get china stocks
    # c_df = pd.read_html('https://www.tradesmax.com/component/k2/item/4480-chinese-stock')
    headers = {
        "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36"
    }
    url = "https://www.tradesmax.com/component/k2/item/4480-chinese-stock"
    r = requests.get(url, headers=headers)
    c_df = pd.read_html(r.content, thousands=None, decimal=",")

    c_tickers_list = c_df[0]["代码"].to_list()
    c_tickers_list.extend(
        [
            "XPEV",
            "LI",
            "BEKE",
            "DIDIY",
            "ZH",
            "TME",
            "GOTU",
            "LI",
            "NIO",
            "WB",
            "EDU",
            "TAL",
            "PDD",
            "TCEHY",
        ]
    )
    c_web_df = pd.DataFrame(columns=web_df.columns)
    c_web_df["Symbol"] = c_tickers_list
    c_web_df["Security"] = c_tickers_list
    c_web_df["GICS Sector"] = "China CDR"
    c_web_df["GICS Sub-Industry"] = "China CDR"

    web_df = pd.concat([web_df, c_web_df], axis=0)
    tickers_list = list(web_df["Symbol"])
    tickers_list = list(set(tickers_list) - set(tickers_delisted))
    tickers_list = tickers_list[:]
    # print('tickers_list',len(tickers_list))

    return tickers_list, web_df


def getERdate2(symbol):
    finnhub_client = finnhub.Client(api_key=FINNHUB_KEY)
    try:
        er_dates = finnhub_client.earnings_calendar(
            _from=date.today(),
            to=date.today() + timedelta(days=100),
            symbol=symbol,
            international=False,
        )
    except Exception as ex:
        print (ex)
        er_dates = {}
    if not er_dates or not er_dates["earningsCalendar"]:
        return "2099-12-31"
    else:
        return er_dates["earningsCalendar"][-1]["date"]


def send_email_html(body_html):
    recipients = ["omnimahui@gmail.com"]
    emaillist = [elem.strip().split(",") for elem in recipients]
    msg = MIMEMultipart()
    msg["Subject"] = "Earning Calendar"
    msg["From"] = "omnimahui@gmail.com"

    html = """\
    <html>
      <head></head>
      <body>
        {0}
      </body>
    </html>
    """.format(
        body_html
    )
    part1 = MIMEText(html, "html")
    msg.attach(part1)

    smtp = smtplib.SMTP(SMTP_SERVER, port=SMTP_PORT)
    smtp.ehlo()  # send the extended hello to our server
    smtp.starttls()  # tell server we want to communicate with TLS encryption
    smtp.login(SMTP_USER, SMTP_PASS)  # login to our email server

    # send our email message 'msg' to our boss
    smtp.sendmail(msg["From"], emaillist, msg.as_string())

    smtp.quit()  # finally, don't forget to close the connection
    return

symbols, web_df = get_sp500()
calendar_dict = dict()
for i in symbols:
    calendar_dict[i] = getERdate2(i)
    time.sleep(1)
df = pd.DataFrame(calendar_dict.items(), columns=["Symbol", "Index"]).set_index("Index")
df = df.loc[df.index != "2099-12-31"]
df = df.groupby(df.index).agg(lambda x: x.tolist())
df = df.applymap(lambda x: str(x).replace("'", ""))
df.style.set_table_styles(
    [
        dict(selector="th", props=[("text-align", "left")]),
        dict(selector="td", props=[("text-align", "left"), ("width", "640px")]),
    ]
)
send_email_html(build_table(df.reset_index(), "blue_light"))

In [12]:
df.style.set_table_styles(
    [
        dict(selector="th", props=[("text-align", "left")]),
        dict(selector="td", props=[("text-align", "left"), ("width", "640px")]),
    ]
)

,Symbol
Index,
2025-02-06,"[BMY, APD, FE, HII, LLY, EFX, HSY, AMZN, CPT, MKTX, YUM, XEL, ICE, MPWR, MCHP, MTD, EXPE, REG, PM, COP, IQV, MHK, HLT, K, SNA, BWA, HON, BDX, ZBH, CMS, LH, RL, VRSN, KVUE, FTNT, PFG, LIN, TTWO, TPR]"
2025-02-07,"[KIM, FTV, CBOE]"
2025-02-10,"[INCY, ROK, CINF, VRTX, L, ACGL, ON, ANET, MCD]"
2025-02-11,"[AIZ, FIS, MAR, MAS, WELL, SMCI, SPGI, ECL, EW, AIG, HUM, GILD, LDOS, KO, DD, CARR]"
2025-02-12,"[ALB, NI, TYL, KHC, CSCO, EQIX, PARA, ROL, WMB, EXC, D, VTR, WAB, SW, PAYC, CME, BIIB, GNRC, CVS, ES, MLM, WAT, IPG, MGM]"
2025-02-13,"[GPN, DXCM, GEHC, PANW, AEE, TAP, RSG, DVA, GDDY, HWM, AMAT, FRT, ABNB, MCO, DLR, WYNN, DUK, DTE, ZBRA, AEP, IR, PCG, CBRE, WST, DE, PPL, POOL, MSI, IRM, ZTS]"
2025-02-14,[MRNA]
2025-02-15,[AZO]
2025-02-18,"[GPC, OXY, CSGP, BIDU, IFF, CDNS, ETR, VMC, CE, DVN, EQT, EXPD, ALLE, MDT]"
